Поскольку в интернете я не нашел готовых датасетов с запросами на бронирование билетов или отелей, то я решил их сгенерировать при помощи LLM. Для этого собрал следующие данные:
- список городов мира
- список отелей мира

Сначала генерируется некоторое кол-во json файлов для каждого из классов задач.
Для покупки билетов генерируются следующие поля:
- Город отправления (случайный элемент из списка городов)
- Город назначения (случайный элемент из списка городов)
- Дата вылета (случайная дата)
- Дата возврата (Дата вылета + n дней)

Для бронирования отелей генерируются следующие поля:
- Город (случайный элемент из списка городов)
- Отель (случайный элемент из списка отелей)
- Дата заезда (случайная дата)
- Кол-во гостей (n человек)
- Время пребывания (n дней)

Потом каждый json подается в LLM с задачей сгенерировать запрос клиента, используя данные из json. Настраивается температура и параметры семплирования для большей разнообразности данных.

# Подготовка данных

In [1]:
import pandas as pd
import json

# Загрузка списка городов
with open('Data/cities.json') as f:
    cities = json.load(f)

city_names = [city['name'] for city in cities['city']]

# Загрузка списка отелей
hotels_df = pd.read_csv('Data/hotels.csv', encoding_errors='ignore')
hotel_names = list(hotels_df[' HotelName'].values)

print(f'Hotels: {len(hotel_names)}\nCities: {len(city_names)}')

Hotels: 1010033
Cities: 10969


# Генерация json для самолетов

In [2]:
from random import sample, randrange
import datetime

def generate_random_date():
    """
    Генерирует случайную дату между 01-01-2023 и 31-12-2024
    """
    start_date = datetime.date(2023, 1, 1)
    end_date = datetime.date(2024, 12, 31)
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = randrange(days_between_dates)
    random_date = start_date + datetime.timedelta(days=random_number_of_days)
    
    return random_date

def generate_tickets(city_names):
    """
    Генерирует json для билетов на самолет
    """
    sample_city = sample(city_names, 2)
    departure_date = generate_random_date()
    return_date = departure_date + datetime.timedelta(days=randrange(1,30))
    return {
        'departure_city': sample_city[0],
        'arrival_city': sample_city[1],
        'departure_date': departure_date.strftime('%d-%m-%Y'),
        'return_date': return_date.strftime('%d-%m-%Y')
    }
generate_tickets(city_names)

{'departure_city': 'Пицунда',
 'arrival_city': 'Триест',
 'departure_date': '03-05-2024',
 'return_date': '27-05-2024'}

# Генерация json для отелей

In [3]:
def generate_booking(city_names, hotel_names):
    """
    Генерирует json для отелей
    """
    sample_city = sample(city_names, 2)
    departure_date = generate_random_date()
    return_date = departure_date + datetime.timedelta(days=randrange(1,30))
    return {
        'city': sample(city_names, 1)[0],
        'hotel': sample(hotel_names, 1)[0],
        'date': generate_random_date().strftime('%d-%m-%Y'),
        'guests': randrange(1, 5),
        'days': randrange(1, 30)
    }
generate_booking(city_names, hotel_names)

{'city': 'Сосновец',
 'hotel': 'Gasthaus Zum Schwan',
 'date': '13-04-2024',
 'guests': 4,
 'days': 12}

# Генерация датасета
Сгенерируем по 1000 примеров для каждого типа задачи

In [4]:
n_samples = 1000
data = {
    'tickets': [],
    'booking': []
}
for i in range(n_samples):
    data['tickets'].append(
        {
            'json': generate_tickets(city_names),
            'text': ''
        }
    )
    data['booking'].append(
        {
            'json': generate_booking(city_names, hotel_names),
            'text': ''
        }
    )

# Загрузка llm
Загрузим llm сразу в vllm, чтобы быстрей сгенерировать данные

In [5]:
from vllm import LLM, SamplingParams
from tqdm import tqdm

llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.2")

INFO 04-07 14:25:05 llm_engine.py:70] Initializing an LLM engine with config: model='mistralai/Mistral-7B-Instruct-v0.2', tokenizer='mistralai/Mistral-7B-Instruct-v0.2', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)
INFO 04-07 14:25:12 llm_engine.py:275] # GPU blocks: 12691, # CPU blocks: 2048
INFO 04-07 14:25:13 model_runner.py:501] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-07 14:25:13 model_runner.py:505] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode.
INFO 04-07 14:25:16 model_runner.py:547] Graph capturing finished in 3 se

In [6]:
# Генерация текстов для бронирования отелей
sampling_params = SamplingParams(temperature=1.2, top_p=0.5, max_tokens=300, stop=['\n\n'])
n_samples = 1000
i = 0

with tqdm(total=n_samples) as pbar:
    while i < n_samples:
        sample = data['booking'][i]['json']
        booking_prompt = f"""
        Представь, что ты русский человек, который желает забронировать номер в отеле. Напиши короткое сообщение в чат. Используй следующие параметры:
        
        Отель: {sample['hotel']}
        """
    
        # С вероятностью 20% не используем доступные данные
        if randrange(1, 11) <= 8:
            booking_prompt += f"\nГород: {sample['city']}"
        else:
            data['booking'][i]['json']['city'] = ''
            
        if randrange(1, 11) <= 8:
            booking_prompt += f"\nДата заселения: {sample['date']}"
        else:
            data['booking'][i]['json']['date'] = ''
            
        if randrange(1, 11) <= 8:
            booking_prompt += f"\nКоличество дней проживания: {sample['days']}"
        else:
            data['booking'][i]['json']['days'] = ''
            
        if randrange(1, 11) <= 8:
            booking_prompt += f"\nКоличество гостей: {sample['guests']}"
        else:
            data['booking'][i]['json']['guests'] = ''
            
        booking_prompt += '\n\nОтвет:'
        
        outputs = llm.generate(booking_prompt, sampling_params, use_tqdm=False)
        
        # Print the outputs.
        for output in outputs:
            prompt = output.prompt
            generated_text = output.outputs[0].text
        data['booking'][i]['text'] = generated_text
        if generated_text != '':
            pbar.update(1)
            i += 1

100%|██████████| 1000/1000 [29:54<00:00,  1.79s/it]


In [9]:
# Генерация текстов для авиабилетов
sampling_params = SamplingParams(temperature=1.2, top_p=0.5, max_tokens=300, stop=['\n\n'])
n_samples = 1000
i = 0

with tqdm(total=n_samples) as pbar:
    while i < n_samples:
        sample = data['tickets'][i]['json']
        tickets_prompt = f"""
        Представь, что ты русский человек, который желает купить билет на самолет. Напиши короткое сообщение в чат. Используй следующие параметры:
        
        Город отправления: {sample['departure_city']}
        Город прибытия: {sample['arrival_city']}
        """
    
        # С вероятностью 20% не используем доступные данные
        if randrange(1, 11) <= 8:
            tickets_prompt += f"\nДата отправления: {sample['departure_date']}"
        else:
            data['tickets'][i]['json']['departure_date'] = ''
            
        if randrange(1, 11) <= 8:
            tickets_prompt += f"\nДата возвращения: {sample['return_date']}"
        else:
            data['tickets'][i]['json']['return_date'] = ''
            
        tickets_prompt += '\n\nОтвет:'
        
        outputs = llm.generate(tickets_prompt, sampling_params, use_tqdm=False)
        
        # Print the outputs.
        for output in outputs:
            prompt = output.prompt
            generated_text = output.outputs[0].text
        data['tickets'][i]['text'] = generated_text
        if generated_text != '':
            pbar.update(1)
            i += 1

100%|██████████| 1000/1000 [21:56<00:00,  1.32s/it]


In [10]:
with open("Data/generated_dataset.json", "w") as outfile: 
    json.dump(data, outfile, ensure_ascii=False)

# Преобразование в DataFrame

In [11]:
import json
import pandas as pd

with open('Data/generated_dataset.json', 'r') as file:
    data = json.load(file)

tickets = data['tickets']
booking = data['booking']

raw_df = {
    'json': [],
    'text': [],
    'label': []
}

for ticket in tickets:
    raw_df['json'].append(ticket['json'])
    raw_df['text'].append(ticket['text'])
    raw_df['label'].append('самолет')

for book in booking:
    raw_df['json'].append(book['json'])
    raw_df['text'].append(book['text'])
    raw_df['label'].append('отель')

df = pd.DataFrame(raw_df)
df.to_csv('Data/generated_data.csv', index=False)